In [7]:
!pip install vidgear

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.4 MB/s eta 0:00:00


In [8]:
# Import library
import torch

import cv2
import numpy as np
import math
import functools
from vidgear.gears import WriteGear

In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

Cloning into 'yolov5'...
remote: Enumerating objects: 15814, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 15814 (delta 10), reused 25 (delta 2), pack-reused 15768
Receiving objects: 100% (15814/15814), 14.64 MiB | 30.29 MiB/s, done.
Resolving deltas: 100% (10819/10819), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 8.7 MB/s eta 0:

In [4]:
# Load model
character_recognition = torch.hub.load('ultralytics/yolov5', 'custom', path = './Weight/character_recognition.pt')
plate_detection = torch.hub.load('ultralytics/yolov5', 'custom', path = './Weight/plate_detection.pt')


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-19 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)



[Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/cycler-0.11.0.dist-info/METADATA'


Fusing layers... 
Model summary: 157 layers, 7091035 parameters, 0 gradients, 16.0 GFLOPs
Adding AutoShape... 
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-19 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)



[Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/idna-3.4.dist-info/METADATA'


Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [6]:
def compare(rect1, rect2):
    if abs(rect1[1] - rect2[1]) > 20:
        return rect1[1] - rect2[1]
    else:
        return rect1[0] - rect2[0]

def linearEquation(p1, p2):
    if p1[0] == p2[0]:
        p1[0] = p2[0] + 1

    a = (p1[1] - p2[1]) / (p1[0] - p2[0])
    b = p1[1] - a * p1[0]
    return a, b

def rotateByAngle(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rotation_matrix = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rotation_matrix, image.shape[1::-1])
    return result

def rotateImage(image):
    edges_image = cv2.Canny(image,  threshold1 = 100,  threshold2 = 200, apertureSize = 3, L2gradient = True)
    lines = cv2.HoughLinesP(edges_image, 1, math.pi/180, threshold=40, minLineLength=image.shape[1]/2.6, maxLineGap=image.shape[1]/25)

    if lines is not None:
        list_angle = []
        for [[x1, y1, x2, y2]] in lines:
            a, b = linearEquation([x1, y1], [x2, y2])
            if a > 0:
                angle = math.atan(a) / math.pi * 180
            else:
                angle = -math.atan(-a) / math.pi * 180
            if abs(angle) <= 45:
                list_angle.append(angle)

        if not list_angle:
            return image

        angle = sum(list_angle) / len(list_angle)
        image = rotateByAngle(image, angle)

    return image

In [9]:

cap = cv2.VideoCapture('./test.mp4')
out = WriteGear(output= './Output.mp4')

while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    print("Can't receive frame (stream end?). Exiting ...")
    break

  detection = plate_detection(frame)
  detect_results = detection.pandas().xyxy[0].to_numpy()

  for i in detect_results:
        if i[4] >= 0.6:
            x_min = int(i[0])
            x_max = int(i[2])
            y_min = int(i[1])
            y_max = int(i[3])

            img = frame[y_min:y_max, x_min:x_max]
            img = rotateImage(img)
            detect_char = character_recognition(img)
            table = detect_char.pandas().xyxy[0].to_numpy()
            if len(table) == 8 or len(table) == 9 :
              boundingBoxes = sorted(table, key=functools.cmp_to_key(compare) )
              character = [i[6] for i in boundingBoxes]
              character = ''.join(character)
              cv2.rectangle(frame, (x_min, y_min), (x_max,y_max), (255,0,0), 2)
              cv2.putText(frame, character, (x_min, y_min), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

  out.write(frame)
  if cv2.waitKey(1) == ord('q'):
    break

cap.release()
out.close()
cv2.destroyAllWindows()



Can't receive frame (stream end?). Exiting ...
